# DX 704 Week 7 Project

This week's project will investigate issues in a quadcopter controller based using a linear quadratic regulator.
You will start with an existing model of the system and logs from a quadcopter based on it, investigate discrepancies, and ultimately train a new model of the system dynamics.

The full project description and a template notebook are available on GitHub: [Project 7 Materials](https://github.com/bu-cds-dx704/dx704-project-07).


## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Introduction

You've just joined a drone startup.
On your first day, you join your new team to watch a test flight for a new quadcopter prototype.
Watching the prototype fly, the team comments that it is not as smooth as usual and suspects that something is off in the controller.
They provide you a copy of the dynamics model and log data from the prototype to investigate.

The quadcopter control model is a slightly more sophisticated version of the 1D quadcopter problem previously considered.

The state vector $\mathbf{x}_t$ now includes an acceleration component, and the action vector now has a servo control for the throttle instead of a raw force component.
\begin{array}{rcl}
\mathbf{x}_t & = & \begin{bmatrix} x_t \\ v_t \\ a_t \end{bmatrix} \\
\mathbf{u_t} & = & \begin{bmatrix} u_t \end{bmatrix}
\end{array}

## Part 1: Reconstruct the Control Matrix

You are provided the dynamics model in the files `model-A.tsv`, `model-B.tsv`, `cost-Q.tsv` and `cost-R.tsv`.
Recompute the control matrix $\mathbf{K}$ to be used in the infinite horizon linear quadratic regulator problem.

In [1]:
# YOUR CHANGES HERE
import control
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

A = pd.read_csv("model-A.tsv", sep="\t").set_index("index")
B = pd.read_csv("model-B.tsv", sep="\t").set_index("index")
Q = pd.read_csv("cost-Q.tsv", sep="\t").set_index("index")
R = pd.read_csv("cost-R.tsv", sep="\t").set_index("index")
qc_log = pd.read_csv("qc-log.tsv", sep="\t").set_index("index")
qc_train = pd.read_csv("qc-train.tsv", sep="\t").set_index("index")

K, P, _ = control.dlqr(A, B, Q, R)
K_vec = np.asarray(K).ravel()

qc_log

,x,v,a,u
index,,,,
0,-5.000000e+00,0.000000e+00,0.000000e+00,1.702188e+00
1,-5.000000e+00,-1.702188e-02,1.531969e+00,-1.263200e+00
2,-5.018724e+00,1.452683e+00,5.482855e-01,-1.249321e+00
3,-3.420773e+00,1.840779e+00,-5.212749e-01,-2.121274e-01
4,-1.395916e+00,1.163611e+00,-7.643170e-01,4.528954e-01
...,...,...,...,...
95,-1.855920e-18,1.097460e-18,-1.592936e-19,-4.843645e-19
96,-6.487133e-19,8.412290e-19,-6.111510e-19,3.189634e-19
97,2.766386e-19,1.733230e-19,-3.851990e-19,4.316642e-19


Save $\mathbf{K}$ in a file "control-K-intended.tsv" with columns x, v and a.

In [2]:
# YOUR CHANGES HERE

df_K = pd.DataFrame([K_vec], columns=["x", "v", "a"])
df_K.to_csv("control-K-intended.tsv", sep="\t", index=False)



Submit "control-K-intended.tsv" in Gradescope.

## Part 2: Recompute the Actions for the Logged States

You get access to the log data for the prototype as the file "qc-log.tsv".
It conveniently saves all the state and actions made.
Recompute the actions based on your $\mathbf{K}$ matrix computed in part 1.

In [3]:
# YOUR CHANGES HERE

X = qc_log[["x","v","a"]].to_numpy(float)
u_check = -(X @ K_vec.reshape(-1,1)).ravel()


Save your computed actions as "qc-check.tsv" with columns "index" and "u_check".

In [4]:
# YOUR CHANGES HERE

qc_check = pd.DataFrame({
    "index": qc_log.index.to_numpy(),
    "u_check": u_check
})
qc_check.to_csv("qc-check.tsv", sep="\t", index=False)


Submit "qc-check.tsv" in Gradescope.

## Part 3: Reverse Engineer the Actual Control Matrix

Now that you have found a systematic difference between your computed actions and the logged actions, estimate $
$, the control matrix that was actually used to choose actions in the prototype.

Hint: With a linear quadratic regulator, the optimal actions are always linear combinations of the state that are calculated using the control matrix.
You can use linear regression to reverse-engineer the coefficients in the control matrix.

In [5]:
# YOUR CHANGES HERE

u = qc_log["u"].to_numpy(float).reshape(-1, 1)
k_hat, *_ = np.linalg.lstsq(X, u, rcond=None)
K_actual = (-k_hat.ravel())  
u_hat = -(X @ K_actual.reshape(-1,1)).ravel()
mae = np.mean(np.abs(u_hat - u.ravel()))

Save $\mathbf{K}_{\mathrm{actual}}$ in "control-K-actual.tsv" with the same format as "control-K-intended.tsv".

In [6]:
# YOUR CHANGES HERE

df_K_actual = pd.DataFrame([K_actual], columns=["x", "v", "a"])
df_K_actual.insert(0, "index", "u")
df_K_actual.to_csv("control-K-actual.tsv", sep="\t", index=False)

Submit "actual-K.tsv" in Gradescope.

## Part 4: Recompute the System Dynamics from the Log Data

On further investigation, it turns out that the control matrix $\mathbf{K}$ in the prototype was modified to compensate for state prediction errors.
You would like to recompute the $\mathbf{A}$ and $\mathbf{B}$ matrices using the log data since they are used to predict the next states.
However, since the action vector $\mathbf{u}_t$ is linearly dependent on the state via $\mathbf{u}_t=-\mathbf{K} \mathbf{x}_t$, you need a new data set so you can separate the effects of the $\mathbf{A}$ and $\mathbf{B}$ matrices.
Your co-workers kindly provide a new file "qc-train.tsv" where noise was added to each action.
Estimate the true $\mathbf{A}$ and $\mathbf{B}$ matrices based on this file.

In [7]:
# YOUR CHANGES HERE

X_t   = qc_train[["x", "v", "a"]].to_numpy(float)[:-1]
U_t   = qc_train[["u"]].to_numpy(float)[:-1]
X_tp1 = qc_train[["x", "v", "a"]].to_numpy(float)[1:]

phi = np.hstack([X_t, U_t])
theta, *_ = np.linalg.lstsq(phi, X_tp1, rcond=None)

A_new = theta[:3, :].T
B_new = theta[3:, :].T


Save $\mathbf{A}$ and $\mathbf{B}$ to "model-A-new.tsv" and "model-B-new.tsv" respectively.

In [8]:
# YOUR CHANGES HERE

df_A = pd.DataFrame(A_new, index=["x","v","a"], columns=["x","v","a"])
df_A.insert(0, "index", df_A.index)
df_A.to_csv("model-A-new.tsv", sep="\t", index=False)

df_B = pd.DataFrame(B_new, index=["x","v","a"], columns=["u"])
df_B.insert(0, "index", df_B.index)
df_B.to_csv("model-B-new.tsv", sep="\t", index=False)

Submit "model-A-new.tsv" and "model-B-new.tsv" in Gradescope.

## Part 5: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

## Part 6: Acknowledgements

If you discussed this assignment with anyone, please acknowledge them here.
If you did this assignment completely on your own, simply write none below.

If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for. If you did not use any other libraries, simply write none below.

If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy. If you did not use any generative AI tools, simply write none below.

NONE